# Ejercicio 10:  RAG con bases de datos vectorial + text-to-speech

En este ejercicio vamos a aprender a utilizar RAG con bd vectorial y text-to-speech


In [1]:
# Instalación de dependencias
#!pip install google-generativeai faiss-cpu sentence-transformers PyMuPDF pandas numpy nltk IPython pydub pyttsx3 requests
import fitz  # PyMuPDF
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import faiss
from google import genai
from google.genai import types
from IPython.display import Audio, display
from pydub import AudioSegment
import io
import tempfile
import pyttsx3
import requests
import time
from io import StringIO

C:\Users\Christian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Christian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
# Configuración de APIs
GEMINI_API_KEY = "API"
ELEVENLABS_API_KEY = "API"
ELEVENLABS_VOICE_ID = "API"

gemini_client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
# 1. Cargar y procesar el PDF
pdf_path = 'irbook.pdf'
doc = fitz.open(pdf_path)
full_text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    full_text += f"PAGE_{page_num+1}: {page.get_text()}\n"

print(f"Total de páginas del PDF: {len(doc)}")

Total de páginas del PDF: 581


In [4]:
# 2. Limpieza básica del texto
patterns_to_remove = [
    r'Online edition \(c\) \d{4} Cambridge UP',
    r'DRAFT!.*Feedback welcome\.',
    r'Cambridge University Press\. Feedback welcome\.',
    r'Figure \d+\.\d+:.*',
    r'Table \d+\.\d+:.*'
]
clean_text = full_text
for pattern in patterns_to_remove:
    clean_text = re.sub(pattern, '', clean_text, flags=re.IGNORECASE)
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

In [5]:
# 3. Usar Gemini para extraer secciones
def extract_sections(text):
    section_prompt = f"""
    Analiza este texto académico y extrae secciones principales con:
    1. Título exacto
    2. Rango de páginas (ej: 5-7)
    3. Breve resumen
    4. Siempre inicia desde la primera página del documento
    
    Formato:
    SECCION: [título]
    PAGINAS: [inicio-fin] "solo números, NADA de caracteres que no sean números"
    CONTENIDO: [resumen]
    ---
    
    Texto: {text[:500000]}...
    """
    try:
        response = gemini_client.models.generate_content(
            model="gemini-2.5-flash",
            contents=section_prompt
        )
        return response.text
    except Exception as e:
        print(f"Error al extraer secciones: {e}")
        return ""

sections_info = extract_sections(clean_text)
print("Secciones extraídas - Acortado:")
print(sections_info[:50000] + "...")

Secciones extraídas - Acortado:
SECCION: Brief Contents
PAGINAS: 5-5
CONTENIDO: This section lists the main chapters of the book along with their starting page numbers, providing a concise overview of the book's organization.
---
SECCION: Contents
PAGINAS: 7-13
CONTENIDO: This section provides a detailed table of contents, listing all chapters and their subsections, along with references to lists of tables, figures, a table of notation, and the preface.
---
SECCION: List of Tables
PAGINAS: 15-17
CONTENIDO: This section presents a list of all tables included in the document, along with their corresponding page numbers.
---
SECCION: List of Figures
PAGINAS: 19-25
CONTENIDO: This section contains a list of all figures presented in the document, each with its respective page number.
---
SECCION: Table of Notation
PAGINAS: 27-30
CONTENIDO: This section deﬁnes the symbols and notation used throughout the document, providing their meaning and the page number where they are introduced.
---
SEC

In [6]:
def extract_sections(gemini_response):
    sections = []
    current = {}
    
    # Normalizar el texto primero
    normalized_text = gemini_response.replace('**SECTION**:', 'SECCION:').replace('*PAGINAS**:', 'PAGINAS:')
    
    # División por secciones
    parts = [p.strip() for p in normalized_text.split('---') if p.strip()]
    
    for part in parts:
        try:
            lines = [l.strip() for l in part.split('\n') if l.strip()]
            if not lines:
                continue
                
            section = {}
            content_buffer = []
            
            for line in lines:
                if line.startswith('SECCION:'):
                    section['title'] = line.replace('SECCION:', '').strip()
                elif line.startswith('PAGINAS:'):
                    pages = re.sub(r'[^\d-]', '', line.replace('PAGINAS:', ''))
                    if '-' in pages:
                        start, end = map(int, pages.split('-'))
                    else:
                        start = end = int(pages)
                    section.update({
                        'start_page': start,
                        'end_page': end,
                        'pages': f"{start}-{end}"
                    })
                elif line.startswith('CONTENIDO:'):
                    section['summary'] = line.replace('CONTENIDO:', '').strip()
                else:
                    content_buffer.append(line)
            
            # Si no tiene CONTENIDO: pero tiene otras líneas
            if 'summary' not in section and content_buffer:
                section['summary'] = ' '.join(content_buffer)
            
            if 'title' in section and 'pages' in section:
                sections.append(section)
                print(f"✓ Sección: '{section['title']}' (págs {section['pages']})")
            else:
                print(f"⚠ Sección incompleta. Datos: {section}")
                
        except Exception as e:
            print(f"✗ Error procesando sección: {str(e)}")
            continue
    
    print(f"\n✅ Secciones identificadas: {len(sections)}")
    return sections

In [7]:
# 4. Procesar PDF con los rangos
def extract_section_texts(sections, pdf_path):
    doc = fitz.open(pdf_path)
    valid_sections = []
    
    for i, section in enumerate(sections):
        try:
            start = max(0, section['start_page'] - 1)
            end = min(len(doc) - 1, section['end_page'] - 1)
            
            content = []
            for page_num in range(start, end + 1):
                content.append(doc.load_page(page_num).get_text())
            
            if content:
                valid_sections.append({
                    'id': i + 1,
                    'pages': f"{section['start_page']}-{section['end_page']}",
                    'title': section['title'],
                    'content': '\n'.join(content),
                    'summary': section.get('summary', '')
                })
        except Exception as e:
            print(f"Error en sección {section.get('title', '?')}: {str(e)}")
    
    return valid_sections
sections = extract_sections(sections_info)  # sections_info viene del Paso 3
documents = extract_section_texts(sections, pdf_path)  # <- Esto crea la variable 'documents'

✓ Sección: 'Brief Contents' (págs 5-5)
✓ Sección: 'Contents' (págs 7-13)
✓ Sección: 'List of Tables' (págs 15-17)
✓ Sección: 'List of Figures' (págs 19-25)
✓ Sección: 'Table of Notation' (págs 27-30)
✓ Sección: 'Preface' (págs 31-37)
✓ Sección: '1 Boolean retrieval' (págs 39-47)
✓ Sección: '2 The term vocabulary and postings lists' (págs 49-56)
✓ Sección: '3 Dictionaries and tolerant retrieval' (págs 57-65)
✓ Sección: '4 Index construction' (págs 67-83)
✓ Sección: '5 Index compression' (págs 85-107)
✓ Sección: '6 Scoring, term weighting and the vector space model' (págs 109-133)
✓ Sección: '7 Computing scores in a complete search system' (págs 135-150)
✓ Sección: '8 Evaluation in information retrieval' (págs 151-175)
✓ Sección: '9 Relevance feedback and query expansion' (págs 177-194)
✓ Sección: '10 XML retrieval' (págs 195-217)

✅ Secciones identificadas: 16


In [8]:
# 5. Preprocesamiento con NLTK
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_documents(docs):
    processed = []
    for doc in docs:
            tokens = word_tokenize(doc['content'].lower())
            filtered = [
                lemmatizer.lemmatize(t) 
                for t in tokens 
                if t.isalpha() and t not in stop_words
            ]
            doc['processed_text'] = ' '.join(filtered)
            processed.append(doc)
    return processed
processed_documents = preprocess_documents(documents)  # <- Esto crea 'processed_documents'    

In [9]:
# 6. Generación de embeddings
for i, doc in enumerate(processed_documents):
    print(f"Doc {i+1}: ID={doc['id']} | Pages={doc['pages']} | Tokens={len(doc['processed_text'].split())}")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = []
valid_documents = []
for doc in processed_documents:
    if len(doc['processed_text']) >= 10:  # Mínimo 10 caracteres
            emb = embedding_model.encode(doc['processed_text'], show_progress_bar=False)
            embeddings.append(emb)
            valid_documents.append(doc)
            print(f"Embedding generado para ID {doc['id']}")
embeddings_array = np.array(embeddings)

Doc 1: ID=1 | Pages=5-5 | Tokens=84
Doc 2: ID=2 | Pages=7-13 | Tokens=775
Doc 3: ID=3 | Pages=15-17 | Tokens=388
Doc 4: ID=4 | Pages=19-25 | Tokens=924
Doc 5: ID=5 | Pages=27-30 | Tokens=395
Doc 6: ID=6 | Pages=31-37 | Tokens=1576
Doc 7: ID=7 | Pages=39-47 | Tokens=1869
Doc 8: ID=8 | Pages=49-56 | Tokens=1570
Doc 9: ID=9 | Pages=57-65 | Tokens=2161
Doc 10: ID=10 | Pages=67-83 | Tokens=3449
Doc 11: ID=11 | Pages=85-107 | Tokens=4180
Doc 12: ID=12 | Pages=109-133 | Tokens=4766
Doc 13: ID=13 | Pages=135-150 | Tokens=2970
Doc 14: ID=14 | Pages=151-175 | Tokens=4580
Doc 15: ID=15 | Pages=177-194 | Tokens=3534
Doc 16: ID=16 | Pages=195-217 | Tokens=4286
Embedding generado para ID 1
Embedding generado para ID 2
Embedding generado para ID 3
Embedding generado para ID 4
Embedding generado para ID 5
Embedding generado para ID 6
Embedding generado para ID 7
Embedding generado para ID 8
Embedding generado para ID 9
Embedding generado para ID 10
Embedding generado para ID 11
Embedding generado para

In [10]:
# 7. Crear base de datos vectorial con FAISS
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings_array)  # para similitud coseno
index.add(embeddings_array)
print(f"\nBase de datos vectorial creada con {index.ntotal} documentos")



Base de datos vectorial creada con 16 documentos


In [11]:
#8. DataFrame de documentos
df_documents = pd.DataFrame({
    'ID': [doc['id'] for doc in valid_documents],
    'Pag': [doc['pages'] for doc in valid_documents],
    'Doc': [doc['title'] for doc in valid_documents],
    'Contenido_Preprocesado': [doc['processed_text'] for doc in valid_documents],
    'embedding': [emb.tolist() for emb in embeddings_array] if len(embeddings_array) > 0 else [None]*len(valid_documents)
})
df_documents

,ID,Pag,Doc,Contenido_Preprocesado,embedding
0,1,5-5,Brief Contents,online edition c cambridge draft april cambrid...,"[0.005014621652662754, -0.02226206660270691, -..."
1,2,7-13,Contents,online edition c cambridge draft april cambrid...,"[0.05708130821585655, -0.06470365077257156, -0..."
2,3,15-17,List of Tables,online edition c cambridge draft april cambrid...,"[-0.0041187903843820095, -0.01612200029194355,..."
3,4,19-25,List of Figures,online edition c cambridge draft april cambrid...,"[0.03033173829317093, -0.04972848296165466, -0..."
4,5,27-30,Table of Notation,online edition c cambridge draft april cambrid...,"[0.013839115388691425, -0.04511633142828941, -..."
5,6,31-37,Preface,online edition c cambridge draft april cambrid...,"[0.014605063013732433, -0.057978902012109756, ..."
6,7,39-47,1 Boolean retrieval,online edition c cambridge boolean retrieval p...,"[-0.004065768793225288, -0.017198272049427032,..."
7,8,49-56,2 The term vocabulary and postings lists,online edition c cambridge boolean retrieval i...,"[0.0532209537923336, -0.03691587597131729, -0...."
8,9,57-65,3 Dictionaries and tolerant retrieval,online edition c cambridge term vocabulary pos...,"[-0.02966645546257496, -0.058079566806554794, ..."
9,10,67-83,4 Index construction,online edition c cambridge term vocabulary pos...,"[0.04624895006418228, -0.05837007984519005, 0...."


In [12]:
# 9. Procesar consulta
query = "¿Qué es el modelo de espacio vectorial (vector space model) y en qué capítulo se explica?"
#query = "¿Quién es Cristóbal Colón según los documentos?"
# Preprocesar la consulta
query_tokens = word_tokenize(query.lower())
processed_query_tokens = [
    lemmatizer.lemmatize(token) 
    for token in query_tokens 
    if token.isalpha() and token not in stop_words
]
processed_query = " ".join(processed_query_tokens)

# Generar embedding para la consulta
query_embedding = embedding_model.encode(processed_query)
query_embedding = np.array([query_embedding])
faiss.normalize_L2(query_embedding)

In [13]:
# 10. Buscar en la base de datos vectorial
k = min(5, len(valid_documents))  # Asegura no pedir más resultados que documentos existentes
similarities, indices = index.search(query_embedding, k)
results_data = []
for i, (similarity, idx) in enumerate(zip(similarities[0], indices[0])):
    if idx < len(valid_documents):
        doc = valid_documents[idx]
        results_data.append({
            'rank': i + 1,
            'document_id': doc['id'],
            'similarity': similarity,
            'pages': doc['pages'],
            'title': doc['title'],
            'content_preview': doc['processed_text'][50:300] + "..."
        })

df_results = pd.DataFrame(results_data)

In [14]:
df_results 

,rank,document_id,similarity,pages,title,content_preview
0,1,1,0.213532,5-5,Brief Contents,niversity press feedback welcome v brief conte...
1,2,7,0.198803,39-47,1 Boolean retrieval,ir also used facilitate semistructured search ...
2,3,5,0.175337,27-30,Table of Notation,niversity press feedback welcome xxvii table n...
3,4,9,0.152436,57-65,3 Dictionaries and tolerant retrieval,list complex sequence character may encoded o...
4,5,2,0.146369,7-13,Contents,niversity press feedback welcome vii content l...


In [15]:
# 11. Construir contexto con síntesis avanzada
def synthesize_context(results_data, valid_documents):
    # Primero se pide a Gemini que sintetice la información relevante
    raw_context = "\n".join([
        f"DOCUMENTO {i+1} (Páginas: {r['pages']}, Similitud: {r['similarity']:.3f}):\n"
        f"Título: {valid_documents[r['document_id']-1]['title']}\n"
        f"Contenido: {valid_documents[r['document_id']-1]['content'][:30000]}\n"
        for i, r in enumerate(results_data)
    ])
    
    synthesis_prompt = f"""
    Eres un experto en síntesis de información académica. Analiza los siguientes fragmentos 
    de documentos y crea un contexto coherente y bien estructurado para responder a 
    la pregunta del usuario. Sigue estas pautas:
    
    1. Combina información de los diferentes fragmentos de forma natural
    2. Elimina repeticiones y redundancias
    3. Mantén las referencias a páginas y secciones importantes
    4. Conserva los conceptos técnicos clave
    5. Proporciona una estructura lógica al contenido
    
    
    Fragmentos:
    {raw_context}
    
    Pregunta original: {query}
    
    Devuelve SOLO el contexto sintetizado, sin comentarios adicionales.
    """
    
    try:
        response = gemini_client.models.generate_content(
            model="gemini-2.0-flash",
            contents=synthesis_prompt
        )
        return response.text
    except Exception as e:
        print(f"Error al sintetizar contexto: {e}")
        return raw_context  # Fallback al contexto original
context = synthesize_context(results_data, valid_documents)

In [16]:
# 12. Generar respuesta con Gemini
prompt = f"""
Eres un experto en Recuperación de Información. Basado en el siguiente contexto sintetizado:
{context}
Responde esta pregunta en español de manera natural y completa:
{query}
Instrucciones:
1. Proporciona una respuesta bien estructurada y concisa basada en el contexto
2. Menciona las páginas relevantes cuando sea apropiado
3. Evita la literalidad, sintetiza la pregunta del usuario pero responde utilizando el contenido del contexto
4. Si un concepto aparece en múltiples secciones, integra la información
5. Basate unicamente en el contexto proporcionado
6. Si no hay información suficiente, solo indica "No existe información suficiente para responder a la pregunta" y termina la respuesta
no expliques nada mas, esa es la respuesta final.
"""
response = gemini_client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
)
answer = response.text
print("Respuesta a la pregunta ",query," : ")
print(answer)
print("="*80)

# 13.1 Text-to-Speech con pyttsx3 
def local_tts(text, attempt=1):
    try:
        print(f"EJECUTANDO pyttsx3 (Intento {attempt})")
        print(f"{'='*50}")
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        spanish_voice = None
        for voice in voices:
            if 'spanish' in voice.languages or 'es' in voice.languages:
                spanish_voice = voice.id
                break
        if spanish_voice:
            engine.setProperty('voice', spanish_voice)
        engine.setProperty('rate', 150)
        engine.setProperty('volume', 1.0)
        #Limitacion del texto a 1000 caracteres para reproducción
        engine.say(text[:900])
        engine.runAndWait()
        print("Síntesis de voz pyttsx3 completada con éxito")
        return True
    except Exception as e:
        print(f"Error con pyttsx3 (Intento {attempt}): {e}")
        if attempt < 2:
            time.sleep(1)
            return local_tts(text, attempt+1)
        return False
# 13.2 Text-to-Speech con ElevenLabs
def elevenlabs_tts(text, attempt=1):
    try:
        print(f"EJECUTANDO ElevenLabs TTS (Intento {attempt})")
        print(f"{'='*50}")
        response = requests.post(
            f"https://api.elevenlabs.io/v1/text-to-speech/{ELEVENLABS_VOICE_ID}",
            headers={
                "xi-api-key": ELEVENLABS_API_KEY,
                "Content-Type": "application/json"
            },
            json={
                #Limitacion de caracteres para reproducción
                "text": text[:900],
                "voice_settings": {
                    "stability": 0.5,
                    "similarity_boost": 0.75
                }
            }
        )
        if response.status_code == 200:
            audio_bytes = response.content
            display(Audio(audio_bytes, autoplay=True))
            return True
        else:
            print(f"Error en ElevenLabs API (Intento {attempt}): {response.status_code} - {response.text}")
            if attempt < 2:
                time.sleep(1)
                return elevenlabs_tts(text, attempt+1)
            return False
    except Exception as e:
        print(f"Error con ElevenLabs TTS (Intento {attempt}): {e}")
        if attempt < 2:
            time.sleep(1)
            return elevenlabs_tts(text, attempt+1)
        return False

Respuesta a la pregunta  ¿Qué es el modelo de espacio vectorial (vector space model) y en qué capítulo se explica?  : 
El modelo de espacio vectorial (Vector Space Model) es un enfoque fundamental en la recuperación de información que permite representar tanto los documentos como las consultas como vectores en un espacio multidimensional. En este espacio, cada dimensión corresponde a un término del vocabulario (Tabla de Notación, p. xxix). Su propósito principal es clasificar y puntuar documentos según su relevancia para una consulta específica.

La similitud entre documentos y consultas se calcula mediante medidas como el producto punto (sección 6.3.1, p. 120). Este modelo integra conceptos clave como la ponderación de términos y la frecuencia de términos, y también aborda funciones variantes de tf-idf (sección 6.4, p. 126).

Este modelo se explica en el Capítulo 6, titulado "Scoring, term weighting and the vector space model" (p. 109), con un desarrollo más profundo en las secciones 

In [17]:
# EJECUCIÓN DE AMBOS SISTEMAS DE TTS
print("\nINICIANDO PROCESO DE TEXT-TO-SPEECH")
#pyttsx3
local_success = local_tts(answer)
# Pequeña pausa entre sistemas
time.sleep(2)
#ElevenLabs
elevenlabs_success = elevenlabs_tts(answer)


INICIANDO PROCESO DE TEXT-TO-SPEECH
EJECUTANDO pyttsx3 (Intento 1)
Síntesis de voz pyttsx3 completada con éxito
EJECUTANDO ElevenLabs TTS (Intento 1)


In [19]:
#pyttsx3
local_success = local_tts(answer)

EJECUTANDO pyttsx3 (Intento 1)
Síntesis de voz pyttsx3 completada con éxito
